In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel, TfidfModel

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/2023년 10월-11월 프로젝트/토큰화 진행 및 불용어 처리/2023년 기사(완료).csv')
df.drop(columns=['Unnamed: 0'],inplace=True)
df.reset_index(drop=True, inplace=True)
df

,date,url,tokenize_title,tokenize_content
0,2023-01-01T06:00:00,https://n.news.naver.com/mnews/article/421/000...,"['새해', '달라지는', '들', '카카오', '먹통', '방지법', '시행', ...","['데이터', '센터', '다중화', '의무', '콘텐츠', '세액', '공제', ..."
1,2023-01-01T12:50:00,https://n.news.naver.com/mnews/article/008/000...,"['비', '대위', '체제', '끝낸', '카카오', '구상권', '청구', '본...","['카카오', '비상', '대책', '위원회', '체제', '종료', '하고', '..."
2,2023-01-01T15:12:00,https://n.news.naver.com/mnews/article/421/000...,"['뉴스', '잇', '쥬', '카카오', '먹통', '사태', '보상', '등장'...","['카카오', '개인', '비즈니스', '파트너', '구분', '해', '일괄', ..."
3,2023-01-01T15:41:00,https://n.news.naver.com/mnews/article/015/000...,"['속', '속결', '마무리', '새해', '새롭게', '카카오', '비', '대...","['지난해', '초유', '먹통', '사태', '비상', '대책', '위원회', '..."
4,2023-01-01T15:54:00,https://n.news.naver.com/mnews/article/215/000...,"['비', '대위', '끝낸', '카카오', '상금', '청구', '전망']","['지난해', '초유', '먹통', '사태', '빚었던', '카카오', '비', '..."
...,...,...,...,...
5600,2023-10-24T21:40:00,https://n.news.naver.com/mnews/article/056/001...,"['카카오', '법인', '처벌', '강수', '둔', '금감원', '장', '발언...","['카카오', '창업', '김범수', '이사회', '의장', '금융감독원', '공개..."
5601,2023-10-24T22:01:00,https://n.news.naver.com/mnews/article/469/000...,"['카카오', '게임', '직원', '오딘', '업데이트', '정보', '유출', ...","['카카오', '게임', '게임', '업데이트', '정보', '사전', '유출', ..."
5602,2023-10-24T22:29:00,https://n.news.naver.com/mnews/article/009/000...,"['풍전등화', '카카오', '시세', '조작', '벌금형', '땐', '카뱅', ...","['금감원', '칼춤', '예고', '카카오', '운명', '경제', '적', '이..."
5603,2023-10-24T23:01:00,https://n.news.naver.com/mnews/article/052/000...,"['금감원', '장', '카카오', '법인', '처벌', '검토', '이번', '송치']","['이복', '현', '금융감독원장', '엔터테인먼트', '시세', '조종', '의..."


In [ ]:
# 각 'tokenize_title'과 'tokenize_content'의 type이 str이므로 이를 list로 변환하기 위함
def str_list(token):
  str_list = token
  str_list = str_list.replace("[","").replace("]","").replace("'","").replace(" ","")
  result = str_list.split(",")
  return result

In [ ]:
# title에 대한 LDA모델 적용
title_docs = []
for doc in df['tokenize_title']:
  title_docs.append(str_list(doc))
title_docs

[['새해', '달라지는', '들', '카카오', '먹통', '방지법', '시행', '콘텐츠', '세액', '공제'],
 ['비', '대위', '체제', '끝낸', '카카오', '구상권', '청구', '본격', '화'],
 ['뉴스', '잇', '쥬', '카카오', '먹통', '사태', '보상', '등장', '현모', '대표', '최종', '후보', '낙점'],
 ['속', '속결', '마무리', '새해', '새롭게', '카카오', '비', '대위', '체제', '종료'],
 ['비', '대위', '끝낸', '카카오', '상금', '청구', '전망'],
 ['먹통', '사태', '카카오', '비', '대위', '종료', '상대', '소송', '전망'],
 ['먹통', '대란', '카카오', '비', '대위', '종료', '경영', '정상화', '돌입'],
 ['먹통', '피해', '보상', '마련', '카카오', '비', '대위', '체제', '종료'],
 ['이모티콘', '보상', '마련', '카카오', '구상권', '소송', '임박'],
 ['카카오', '대표', '미래', '이니셔티브', '센터', '고문'],
 ['공정위', '카카오', '엔터', '불공정', '혐의', '조사', '마치', '고', '상반기', '심의'],
 ['공정위', '카카오', '엔터', '불공정', '혐의', '조사', '마치', '고', '상반기', '심의'],
 ['용인시', '홈페이지', '네이버', '카카오', '로그인', '하세요'],
 ['먹통', '보상', '완료', '카카오', '경영', '집중', '상대로', '구상권'],
 ['게시판', '카카오', '손', '가정', '쌀', '전달'],
 ['공정위', '카카오', '엔터', '불공정', '혐의', '조사', '마치', '고', '상반기', '심의'],
 ['이모티콘', '카카오', '보상', '책', '국민', '누구', '놀리나', '민심', '레이더'],
 ['네이버', '카카오', '새해', '웹툰

In [ ]:
id2word_title = corpora.Dictionary(title_docs)
print(id2word_title)

Dictionary<6217 unique tokens: ['공제', '달라지는', '들', '먹통', '방지법']...>


In [ ]:
for value in id2word_title:
  print(value, id2word_title[value])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1217 제출
1218 절차
1219 예정
1220 착수
1221 현금
1222 모델
1223 아이
1224 학습
1225 게
1226 사장
1227 최선
1228 관리
1229 빠르게
1230 쉽고
1231 놀금
1232 오피스
1233 유지
1234 제도
1235 투
1236 고젝
1237 라오스
1238 상륙
1239 답정너
1240 전원
1241 회의
1242 한판
1243 바이오
1244 반도체
1245 뽑은
1246 펀드매니저
1247 근거리
1248 만나
1249 배송
1250 플러스
1251 영역
1252 개관
1253 도서관
1254 안산
1255 작은
1256 평화
1257 포레
1258 맥시
1259 치킨
1260 칸
1261 초읽기
1262 사용
1263 재생에너지
1264 전력
1265 충당
1266 기준
1267 차
1268 모임
1269 붙나
1270 토스
1271 통장
1272 결과
1273 안도
1274 건설
1275 부문
1276 한화
1277 클립
1278 금리
1279 꺾이나
1280 한풀
1281 환호
1282 인상
1283 호점
1284 조성
1285 기본
1286 대형
1287 손잡고
1288 싸네
1289 아이엠
1290 중형
1291 타타
1292 문
1293 열어
1294 끝
1295 보인다
1296 웃음꽃
1297 무명
1298 사업가
1299 작가
1300 길안내
1301 들어간
1302 맵
1303 손목
1304 자전거
1305 걸까
1306 타다
1307 합병
1308 닫는다
1309 댓글
1310 요청
1311 유족
1312 이태원
1313 참사
1314 폐쇄
1315 번역
1316 외국어
1317 장벽
1318 해준다
1319 허무
1320 맞
1321 안정
1322 일원
1323 화한다
1324 안전성
1325 일원화
1326 올라
1327 방통위
1328 보호
1329 수준
1330 이용자
1331 사자
1332 외국인
1333 벗는다


In [ ]:
corpus_TDM = []
for doc in title_docs:
  result = id2word_title.doc2bow(doc)
  corpus_TDM.append(result)

corpus_TDM # index, 등장 횟수 (순서는 index 순으로 sorting해서 출력됨)

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1)],
 [(8, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1)],
 [(3, 1),
  (8, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1)],
 [(5, 1),
  (8, 1),
  (12, 1),
  (14, 1),
  (16, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1)],
 [(8, 1), (11, 1), (12, 1), (14, 1), (15, 1), (34, 1), (35, 1)],
 [(3, 1),
  (8, 1),
  (12, 1),
  (14, 1),
  (23, 1),
  (33, 1),
  (35, 1),
  (36, 1),
  (37, 1)],
 [(3, 1),
  (8, 1),
  (12, 1),
  (14, 1),
  (33, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1)],
 [(3, 1),
  (8, 1),
  (12, 1),
  (14, 1),
  (16, 1),
  (22, 1),
  (33, 1),
  (42, 1),
  (43, 1)],
 [(8, 1), (10, 1), (22, 1), (37, 1), (42, 1), (44, 1), (45, 1)],
 [(8, 1), (20, 1), (46, 1), (47, 1), (48, 1), (49, 1)],
 [(8, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (5

In [ ]:
tfidf = TfidfModel(corpus_TDM)
corpus_TFIDF = tfidf[corpus_TDM]

n = 10 # 토픽갯수 - 변경가능
lda = LdaModel(corpus=corpus_TFIDF, id2word=id2word_title, num_topics=n, random_state=100)

for t in lda.print_topics():
  print(t[0],":",t[1])

0 : 0.022*"법인" + 0.015*"현" + 0.014*"적극" + 0.009*"이복" + 0.009*"카뱅" + 0.009*"대표" + 0.008*"뱅크" + 0.007*"등록" + 0.006*"도전" + 0.005*"비명"
1 : 0.030*"혐의" + 0.024*"시세" + 0.024*"조종" + 0.018*"금감원" + 0.015*"심사" + 0.014*"김범수" + 0.012*"위기" + 0.011*"청구" + 0.010*"직원" + 0.009*"대표"
2 : 0.017*"처벌" + 0.016*"센터" + 0.016*"검찰" + 0.014*"데이터" + 0.012*"장" + 0.012*"안산" + 0.009*"개" + 0.008*"약세" + 0.008*"열사" + 0.007*"상대"
3 : 0.019*"의장" + 0.011*"금감원" + 0.011*"송치" + 0.009*"검토" + 0.008*"방해" + 0.008*"첫" + 0.008*"회사" + 0.008*"자체" + 0.008*"이번" + 0.007*"먹통"
4 : 0.061*"의혹" + 0.057*"시세" + 0.057*"조종" + 0.038*"금감원" + 0.024*"김범수" + 0.018*"투자" + 0.016*"리스크" + 0.015*"대표" + 0.013*"조사" + 0.013*"창업"
5 : 0.049*"총괄" + 0.014*"배재현" + 0.013*"고발" + 0.010*"투자" + 0.009*"다음" + 0.008*"대표" + 0.006*"코인" + 0.006*"데이터" + 0.006*"한국" + 0.006*"기술"
6 : 0.030*"포토" + 0.009*"프렌즈" + 0.009*"개미" + 0.006*"하반기" + 0.005*"브레인" + 0.005*"대표" + 0.005*"게임" + 0.005*"머니" + 0.004*"모집" + 0.004*"들"
7 : 0.009*"택시" + 0.009*"대구시" + 0.008*"모빌리티" + 0.007*"크루" + 0.007*"문어"

In [ ]:
# 반복문 돌려서 기사에 토픽 일일이 배정해줘야 함(title 부분)
lda[corpus_TFIDF[0]]

[(0, 0.026011834),
 (1, 0.026018001),
 (2, 0.026045527),
 (3, 0.026026081),
 (4, 0.026018815),
 (5, 0.026069418),
 (6, 0.76572406),
 (7, 0.026028829),
 (8, 0.026020741),
 (9, 0.026036663)]

In [ ]:
for i in range(len(corpus_TFIDF)):
  print(lda[corpus_TFIDF[i]])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[(0, 0.033724423), (1, 0.03372912), (2, 0.033727583), (3, 0.033727128), (4, 0.033723336), (5, 0.033724315), (6, 0.03372811), (7, 0.033723492), (8, 0.69646174), (9, 0.033730764)]
[(0, 0.028785635), (1, 0.028787551), (2, 0.028790092), (3, 0.028786825), (4, 0.028787574), (5, 0.53492206), (6, 0.028785497), (7, 0.23476031), (8, 0.028786486), (9, 0.028807977)]
[(0, 0.02800631), (1, 0.028004771), (2, 0.028024433), (3, 0.028001841), (4, 0.46128714), (5, 0.02800218), (6, 0.028002562), (7, 0.028005796), (8, 0.028001295), (9, 0.31466365)]
[(0, 0.027241664), (1, 0.027242677), (2, 0.027245099), (3, 0.0272427), (4, 0.7548023), (5, 0.027242083), (6, 0.027242273), (7, 0.027250653), (8, 0.027247932), (9, 0.02724263)]
[(0, 0.031217191), (1, 0.031254046), (2, 0.031227103), (3, 0.031234464), (4, 0.031225828), (5, 0.031225579), (6, 0.031219572), (7, 0.031238966), (8, 0.71892345), (9, 0.031233808)]
[(0, 0.027804602), (1, 0.027799353), (2, 0.027809164), (3, 0.027796336), (

In [ ]:
for i in range(len(corpus_TFIDF)):
  dominant_topic = sorted(lda[corpus_TFIDF[i]], key=lambda x: x[1], reverse=True)[0]
  df.loc[i, 'topic(title)'] = dominant_topic[0]
  df.loc[i, 'topic_details(title)'] = dominant_topic[1]
df

,date,url,tokenize_title,tokenize_content,topic(title),topic_details(title)
0,2023-01-01T06:00:00,https://n.news.naver.com/mnews/article/421/000...,"['새해', '달라지는', '들', '카카오', '먹통', '방지법', '시행', ...","['데이터', '센터', '다중화', '의무', '콘텐츠', '세액', '공제', ...",6.0,0.765730
1,2023-01-01T12:50:00,https://n.news.naver.com/mnews/article/008/000...,"['비', '대위', '체제', '끝낸', '카카오', '구상권', '청구', '본...","['카카오', '비상', '대책', '위원회', '체제', '종료', '하고', '...",1.0,0.760566
2,2023-01-01T15:12:00,https://n.news.naver.com/mnews/article/421/000...,"['뉴스', '잇', '쥬', '카카오', '먹통', '사태', '보상', '등장'...","['카카오', '개인', '비즈니스', '파트너', '구분', '해', '일괄', ...",1.0,0.788783
3,2023-01-01T15:41:00,https://n.news.naver.com/mnews/article/015/000...,"['속', '속결', '마무리', '새해', '새롭게', '카카오', '비', '대...","['지난해', '초유', '먹통', '사태', '비상', '대책', '위원회', '...",2.0,0.604431
4,2023-01-01T15:54:00,https://n.news.naver.com/mnews/article/215/000...,"['비', '대위', '끝낸', '카카오', '상금', '청구', '전망']","['지난해', '초유', '먹통', '사태', '빚었던', '카카오', '비', '...",0.0,0.624934
...,...,...,...,...,...,...
5600,2023-10-24T21:40:00,https://n.news.naver.com/mnews/article/056/001...,"['카카오', '법인', '처벌', '강수', '둔', '금감원', '장', '발언...","['카카오', '창업', '김범수', '이사회', '의장', '금융감독원', '공개...",1.0,0.390883
5601,2023-10-24T22:01:00,https://n.news.naver.com/mnews/article/469/000...,"['카카오', '게임', '직원', '오딘', '업데이트', '정보', '유출', ...","['카카오', '게임', '게임', '업데이트', '정보', '사전', '유출', ...",9.0,0.511587
5602,2023-10-24T22:29:00,https://n.news.naver.com/mnews/article/009/000...,"['풍전등화', '카카오', '시세', '조작', '벌금형', '땐', '카뱅', ...","['금감원', '칼춤', '예고', '카카오', '운명', '경제', '적', '이...",3.0,0.342337
5603,2023-10-24T23:01:00,https://n.news.naver.com/mnews/article/052/000...,"['금감원', '장', '카카오', '법인', '처벌', '검토', '이번', '송치']","['이복', '현', '금융감독원장', '엔터테인먼트', '시세', '조종', '의...",3.0,0.450182


In [ ]:
# 토픽별 비중 확인
df['topic(title)'].unique()

array([6., 1., 2., 0., 8., 3., 5., 7., 9., 4.])

In [ ]:
df['topic(title)'].value_counts()

9.0    735
1.0    707
4.0    699
2.0    632
5.0    555
8.0    527
7.0    485
6.0    468
3.0    450
0.0    347
Name: topic(title), dtype: int64

In [ ]:
# content에 대한 LDA모델 적용
content_docs = []
for doc in df['tokenize_content']:
  content_docs.append(str_list(doc))
content_docs

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
id2word_content = corpora.Dictionary(content_docs)
print(id2word_content)

Dictionary<32881 unique tokens: ['가능해진다', '간섭', '간소화', '갖추면', '같은']...>


In [ ]:
for value in id2word_content:
  print(value, id2word_content[value])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
27881 뚝뚝
27882 말까
27883 말씀드리자면
27884 멀죠
27885 미비했다고
27886 살펴보면요
27887 수사의뢰
27888 아니나
27889 업도
27890 올랐는데요
27891 올리려면
27892 잠잠한가
27893 재위
27894 짤막하게
27895 탁하는
27896 했더니
27897 스며든다
27898 대출받았다
27899 박준석
27900 논개
27901 부울경
27902 산태
27903 진주
27904 김광신
27905 구글지도
27906 꼽았으
27907 넓지
27908 뒤진
27909 분야였는데
27910 우월했다
27911 종결자
27912 치율
27913 냄으로써
27914 건강검진
27915 단양
27916 단양군
27917 디아
27918 받기라도
27919 사카모토
27920 신월동
27921 영화음악
27922 오은지
27923 자재
27924 정진욱
27925 제천
27926 제천시
27927 찬송
27928 최영준
27929 충주시
27930 충청북도
27931 타계
27932 티젠
27933 해드렸는데요
27934 해드렸습니다
27935 글래스
27936 네스
27937 다비
27938 로더
27939 로레알
27940 마르
27941 메종
27942 바뇨
27943 반품
27944 부르켓
27945 시세이도
27946 시슬리
27947 영자
27948 용이하면서도
27949 이병희
27950 정품
27951 직매
27952 꾸미기로
27953 끝말잇기
27954 나빠졌을
27955 늦었지만
27956 돌아보면
27957 뚜렷하게
27958 뚜렷해
27959 무겁지만
27960 시달렸던
27961 아니어도
27962 유력하지만
27963 즐기기
27964 착한다면
27965 팅할
27966 퍼뜨릴
27967 품겠다는
27968 활발해지면
27969 개그우먼
27970 구워내어
27971 놀라는
27972 놀라로
27973 놀라를
27974 누리시길

In [ ]:
corpus_TDM = []
for doc in content_docs:
  result = id2word_content.doc2bow(doc)
  corpus_TDM.append(result)

corpus_TDM # index, 등장 횟수 (순서는 index 순으로 sorting해서 출력됨)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# 토큰화 진행 및 불용어 처리에서 불용어에 대해 업데이트 해줘야함!!!
tfidf = TfidfModel(corpus_TDM)
corpus_TFIDF = tfidf[corpus_TDM]

n = 10 # 토픽갯수 - 변경가능
lda = LdaModel(corpus=corpus_TFIDF, id2word=id2word_content, num_topics=n, random_state=100)

for t in lda.print_topics():
  print(t[0],":",t[1])

0 : 0.002*"팁" + 0.002*"평위" + 0.001*"메일" + 0.001*"택시" + 0.001*"학생증" + 0.001*"하이브" + 0.001*"언론사" + 0.001*"공개매수" + 0.001*"관광" + 0.001*"가맹"
1 : 0.004*"김범수" + 0.003*"센터" + 0.003*"김" + 0.003*"노조" + 0.002*"소송" + 0.002*"뱅크" + 0.002*"창업" + 0.002*"제보" + 0.002*"성남시" + 0.002*"차"
2 : 0.003*"매출" + 0.002*"영업" + 0.002*"이익" + 0.002*"동기" + 0.002*"기사" + 0.002*"감소" + 0.001*"대비" + 0.001*"추천" + 0.001*"수산물" + 0.001*"증가"
3 : 0.014*"조종" + 0.012*"시세" + 0.011*"금감원" + 0.010*"혐의" + 0.008*"의혹" + 0.008*"엔터" + 0.008*"특사" + 0.008*"자본시장" + 0.008*"하이브" + 0.008*"공개매수"
4 : 0.005*"주가" + 0.003*"증권" + 0.003*"김" + 0.003*"하락" + 0.003*"리스크" + 0.003*"의장" + 0.002*"에스엠" + 0.002*"거래" + 0.002*"투자" + 0.002*"뱅크"
5 : 0.005*"센터" + 0.005*"데이터" + 0.003*"여의도" + 0.003*"클라우드" + 0.003*"안산" + 0.003*"서비스" + 0.003*"이니셔티브" + 0.003*"기술" + 0.002*"엔터프라이즈" + 0.002*"모델"
6 : 0.003*"출석" + 0.003*"집회" + 0.003*"의장" + 0.002*"대표" + 0.002*"게임" + 0.002*"배재현" + 0.002*"스코어" + 0.002*"장" + 0.002*"사업" + 0.002*"프렌즈"
7 : 0.005*"압수수색" + 0.004*"영등포구" + 0.003*"페이" + 0.0

In [ ]:
# 반복문 돌려서 기사에 토픽 일일이 배정해줘야 함(content 부분)
lda[corpus_TFIDF[0]]

[(1, 0.27969688), (5, 0.52173537), (9, 0.13104491)]

In [ ]:
for i in range(len(corpus_TFIDF)):
  print(lda[corpus_TFIDF[i]])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[(0, 0.010060563), (1, 0.010063147), (2, 0.01006208), (3, 0.01006203), (4, 0.010064078), (5, 0.28399986), (6, 0.6354962), (7, 0.010070442), (8, 0.010060888), (9, 0.010060705)]
[(0, 0.012116389), (1, 0.7666252), (2, 0.012119649), (3, 0.012118815), (4, 0.012119838), (5, 0.012122554), (6, 0.012117999), (7, 0.1364196), (8, 0.012118656), (9, 0.012121271)]
[(4, 0.9179013)]
[(0, 0.01113399), (1, 0.23035789), (2, 0.01114494), (3, 0.011137636), (4, 0.14617653), (5, 0.011148671), (6, 0.011141624), (7, 0.5454722), (8, 0.01113726), (9, 0.011149259)]
[(5, 0.14816256), (6, 0.7899058)]
[(0, 0.010096376), (1, 0.0100986855), (2, 0.010098207), (3, 0.010097664), (4, 0.48416045), (5, 0.010100278), (6, 0.4350453), (7, 0.01010944), (8, 0.010097057), (9, 0.010096542)]
[(4, 0.24581951), (5, 0.16452879), (6, 0.52380115)]
[(0, 0.010144834), (1, 0.010145225), (2, 0.010144058), (3, 0.010144383), (4, 0.33237508), (5, 0.58646494), (6, 0.010146866), (7, 0.010145939), (8, 0.0101443

In [ ]:
for i in range(len(corpus_TFIDF)):
  dominant_topic = sorted(lda[corpus_TFIDF[i]], key=lambda x: x[1], reverse=True)[0]
  df.loc[i, 'topic(content)'] = dominant_topic[0]
  df.loc[i, 'topic_details(content)'] = dominant_topic[1]
df

,date,url,tokenize_title,tokenize_content,topic(title),topic_details(title),topic(content),topic_details(content)
0,2023-01-01T06:00:00,https://n.news.naver.com/mnews/article/421/000...,"['새해', '달라지는', '들', '카카오', '먹통', '방지법', '시행', ...","['데이터', '센터', '다중화', '의무', '콘텐츠', '세액', '공제', ...",6.0,0.765730,5.0,0.521943
1,2023-01-01T12:50:00,https://n.news.naver.com/mnews/article/008/000...,"['비', '대위', '체제', '끝낸', '카카오', '구상권', '청구', '본...","['카카오', '비상', '대책', '위원회', '체제', '종료', '하고', '...",1.0,0.760566,1.0,0.477518
2,2023-01-01T15:12:00,https://n.news.naver.com/mnews/article/421/000...,"['뉴스', '잇', '쥬', '카카오', '먹통', '사태', '보상', '등장'...","['카카오', '개인', '비즈니스', '파트너', '구분', '해', '일괄', ...",1.0,0.788783,1.0,0.737517
3,2023-01-01T15:41:00,https://n.news.naver.com/mnews/article/015/000...,"['속', '속결', '마무리', '새해', '새롭게', '카카오', '비', '대...","['지난해', '초유', '먹통', '사태', '비상', '대책', '위원회', '...",2.0,0.604431,5.0,0.453324
4,2023-01-01T15:54:00,https://n.news.naver.com/mnews/article/215/000...,"['비', '대위', '끝낸', '카카오', '상금', '청구', '전망']","['지난해', '초유', '먹통', '사태', '빚었던', '카카오', '비', '...",0.0,0.624934,5.0,0.471493
...,...,...,...,...,...,...,...,...
5600,2023-10-24T21:40:00,https://n.news.naver.com/mnews/article/056/001...,"['카카오', '법인', '처벌', '강수', '둔', '금감원', '장', '발언...","['카카오', '창업', '김범수', '이사회', '의장', '금융감독원', '공개...",1.0,0.390883,1.0,0.556160
5601,2023-10-24T22:01:00,https://n.news.naver.com/mnews/article/469/000...,"['카카오', '게임', '직원', '오딘', '업데이트', '정보', '유출', ...","['카카오', '게임', '게임', '업데이트', '정보', '사전', '유출', ...",9.0,0.511587,7.0,0.564752
5602,2023-10-24T22:29:00,https://n.news.naver.com/mnews/article/009/000...,"['풍전등화', '카카오', '시세', '조작', '벌금형', '땐', '카뱅', ...","['금감원', '칼춤', '예고', '카카오', '운명', '경제', '적', '이...",3.0,0.342337,3.0,0.467073
5603,2023-10-24T23:01:00,https://n.news.naver.com/mnews/article/052/000...,"['금감원', '장', '카카오', '법인', '처벌', '검토', '이번', '송치']","['이복', '현', '금융감독원장', '엔터테인먼트', '시세', '조종', '의...",3.0,0.450182,1.0,0.492870


In [ ]:
# 토픽별 비중 확인
df['topic(content)'].unique()

array([5., 1., 6., 7., 8., 4., 9., 0., 2., 3.])

In [ ]:
df['topic(content)'].value_counts()

5.0    1170
6.0    1050
4.0     954
3.0     801
1.0     738
7.0     709
8.0     102
2.0      38
9.0      34
0.0       9
Name: topic(content), dtype: int64

In [ ]:
# CSV 파일로 저장
csv_filename = "news_2023.csv"
df.to_csv(csv_filename, index=False)